In [1]:
import pydae.build_cffi as db
from pydae.bmapu import bmapu_builder
from pydae.build_v2 import builder


## PV plant

![Alt text](pv_nm.svg)

A PV with converted connected to an infinite bus through a line.


![Alt text](spvib.svg)
![Alt text](pv_module.svg)



### Option 1: using a python dic to define the system

In [2]:
M = 1
N = 2

S_pv_mva = 1.0

data = {
    "system":{"name":f"pv_{M}_{N}","S_base":100e6,"K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},
    "buses":[
        {"name":"POI_MV","P_W":0.0,"Q_var":0.0,"U_kV":20.0},
        {"name":   "POI","P_W":0.0,"Q_var":0.0,"U_kV":132.0},
        {"name":  "GRID","P_W":0.0,"Q_var":0.0,"U_kV":132.0}
    ],
    "lines":[
        {"bus_j":"POI_MV","bus_k": "POI","X_pu":0.05,"R_pu":0.0,"Bs_pu":0.0,"S_mva":120},
        {"bus_j":   "POI","bus_k":"GRID","X_pu":0.02,"R_pu":0.0,"Bs_pu":0.0,"S_mva":120, 'sym':True, 'monitor':True}
        ],
    "pvs":[],
    "genapes":[{
          "bus":"GRID","S_n":1000e6,"F_n":50.0,"X_v":0.001,"R_v":0.0,
          "K_delta":0.001,"K_alpha":1e-6}]
    }

for i_m in range(1,M+1):
    name_j = "POI_MV"
    for i_n in range(1,N+1):
        name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
        name_k = 'MV' + name

        data['buses'].append({"name":f"LV{name}","P_W":0.0,"Q_var":0.0,"U_kV":0.4})
        data['buses'].append({"name":f"MV{name}","P_W":0.0,"Q_var":0.0,"U_kV":20.0})

        data['lines'].append({"bus_j":f"LV{name}","bus_k":f"MV{name}","X_pu":0.05,"R_pu":0.0,"Bs_pu":0.0,"S_mva":1.2*S_pv_mva,"monitor":False})
        data['lines'].append({"bus_j":f"{name_k}","bus_k":f"{name_j}","X_pu":0.01,"R_pu":0.01,"Bs_pu":0.0,"S_mva":1.2*S_pv_mva*(N-i_n+1),"monitor":False})
        name_j = name_k
        data['pvs'].append({"bus":f"LV{name}","type":"pv_dq","S_n":S_pv_mva*1e6,"U_n":400.0,"F_n":50.0,"X_s":0.1,"R_s":0.01,"monitor":False,
                            "I_sc":8,"V_oc":42.1,"I_mp":3.56,"V_mp":33.7,"K_vt":-0.160,"K_it":0.065,"N_pv_s":25,"N_pv_p":250})
    


### Option 2: using a .json file or a .hjson file

In [3]:
#grid = bmapu_builder.bmapu('spvib.json')
grid = bmapu_builder.bmapu(data)

grid.uz_jacs = False
grid.verbose = True
grid.construct(f'pv_{M}_{N}')

In [5]:
b = builder(grid.sys_dict,verbose=True)
b.dict2system()
b.functions()
b.jacobians()
b.cwrite()
b.template()
b.compile()

2023-06-14 13:56:42,371 check_system
2023-06-14 13:56:42,384 computing jacobians Fx_run,Fy_run
2023-06-14 13:56:42,389 computing jacobians Gx_run,Gy_run


2023-06-14 13:56:42,719 computing jacobians Fu_run,Gu_run
2023-06-14 13:56:42,836 computing jacobians Fx_ini,Fy_ini
2023-06-14 13:56:42,839 computing jacobians Gy_ini
2023-06-14 13:56:42,962 computing jacobians Hx_run,Hy_run,Hu_run
2023-06-14 13:56:43,000 end checking and computing jacobians
2023-06-14 13:56:43,001 f_ini symbolic to c and xyup
2023-06-14 13:56:43,004 end f_ini symbolic to c and xyup
2023-06-14 13:56:43,005 f_run symbolic to c and xyup
2023-06-14 13:56:43,007 end f symbolic to c and xyup
2023-06-14 13:56:43,008 g_ini symbolic to c and xyup
2023-06-14 13:56:43,141 end g_ini symbolic to c and xyup
2023-06-14 13:56:43,142 g_run symbolic to c and xyup
2023-06-14 13:56:43,256 end g_run symbolic to c and xyup
2023-06-14 13:56:43,257 h symbolic to c and xyup
2023-06-14 13:56:43,268 end h symbolic to c and xyup
2023-06-14 13:56:43,268 computing jac_ini
2023-06-14 13:56:43,270 computing jac_run
2023-06-14 13:56:43,271 computing jac_trap
2023-06-14 13:56:43,273 end of large jacob